In [1]:
import torch
from torch import nn
from torch import optim

In [2]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR
import itertools

initial_lr = 0.1

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        pass

net_1 = model()
net_2 = model()

optimizer_1 = torch.optim.Adam(net_1.parameters(), lr = initial_lr)
print("******************optimizer_1*********************")
print("optimizer_1.defaults：", optimizer_1.defaults)
print("optimizer_1.param_groups长度：", len(optimizer_1.param_groups))
print("optimizer_1.param_groups一个元素包含的键：", optimizer_1.param_groups[0].keys())
print()

optimizer_2 = torch.optim.Adam([*net_1.parameters(), *net_2.parameters()], lr = initial_lr)
# optimizer_2 = torch.opotim.Adam(itertools.chain(net_1.parameters(), net_2.parameters())) # 和上一行作用相同
print("******************optimizer_2*********************")
print("optimizer_2.defaults：", optimizer_2.defaults)
print("optimizer_2.param_groups长度：", len(optimizer_2.param_groups))
print("optimizer_2.param_groups一个元素包含的键：", optimizer_2.param_groups[0].keys())
print()

optimizer_3 = torch.optim.Adam([{"params": net_1.parameters()}, {"params": net_2.parameters()}], lr = initial_lr)
print("******************optimizer_3*********************")
print("optimizer_3.defaults：", optimizer_3.defaults)
print("optimizer_3.param_groups长度：", len(optimizer_3.param_groups))
print("optimizer_3.param_groups一个元素包含的键：", optimizer_3.param_groups[0].keys())


******************optimizer_1*********************
optimizer_1.defaults： {'lr': 0.1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': None}
optimizer_1.param_groups长度： 1
optimizer_1.param_groups一个元素包含的键： dict_keys(['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad', 'maximize', 'foreach', 'capturable', 'differentiable', 'fused'])

******************optimizer_2*********************
optimizer_2.defaults： {'lr': 0.1, 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False, 'maximize': False, 'foreach': None, 'capturable': False, 'differentiable': False, 'fused': None}
optimizer_2.param_groups长度： 1
optimizer_2.param_groups一个元素包含的键： dict_keys(['params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad', 'maximize', 'foreach', 'capturable', 'differentiable', 'fused'])

******************optimizer_3*********************
optimizer_3.defaults： {'lr': 0.1, 'be

In [3]:
param = next(net_1.parameters())
type(param)

torch.nn.parameter.Parameter

## model.named_parameters()
返回值是一个生成器，构成的迭代元素是一个元组。包含两个元素：(name.layer, parameters)

In [4]:
for name, _ in net_1.named_parameters(): 
    print(name)  # 果然是这样子

conv1.weight
conv1.bias
conv2.weight
conv2.bias


### LambdaLR(opeimizer, lr_lambda, last_epoch)
 1. optimizer: 要更改的学习率的优化器
 2. lr_lambda (function of list): 根据epoch计算$\lambda$的函数，返回值一个存全部$\lambda$的列表。分别计算各个parameter groups 的学习率更新用到的 $\lambda$
 3. last_epoch(int): 最后一个epoch的index，如果是训练了很多个epoch后中断了，继续训练，这个值就等于加载的模型的epoch。默认-1表示从头开始训练，即从epoch=1开始


In [7]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import LambdaLR

initial_lr = 0.1

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        pass

net_1 = model()

optimizer_1 = torch.optim.Adam(net_1.parameters(), lr = initial_lr)
scheduler_1 = LambdaLR(optimizer_1, lr_lambda=lambda epoch: 1/(epoch+1))

print("初始化的学习率：", optimizer_1.defaults['lr'])

for epoch in range(1, 11):
    # train

    optimizer_1.zero_grad()
    optimizer_1.step()
    print("第%d个epoch的学习率：%f" % (epoch, optimizer_1.param_groups[0]['lr']))
    scheduler_1.step()


初始化的学习率： 0.1
第1个epoch的学习率：0.100000
第2个epoch的学习率：0.050000
第3个epoch的学习率：0.033333
第4个epoch的学习率：0.025000
第5个epoch的学习率：0.020000
第6个epoch的学习率：0.016667
第7个epoch的学习率：0.014286
第8个epoch的学习率：0.012500
第9个epoch的学习率：0.011111
第10个epoch的学习率：0.010000


### torch.optim.lr_scheduler.StepLR
**更新策略:**

每过step_size个epoch，做一次更新：$$new_lr = initial_lr \times \gamma^{epoch // step_size}$$
其中new_lr是得到的新的学习率，initial_lr是初始化的学习率，step_size是参数step_size，$\gamma$是参数$\gamma$

In [9]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
import itertools

initial_lr = 0.1

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        pass

net_1 = model()

optimizer_1 = torch.optim.Adam(net_1.parameters(), lr=initial_lr)
scheduler_1 = StepLR(optimizer_1, step_size=3, gamma=0.1)

print("初始化的学习率: ", optimizer_1.defaults['lr'])
for epoch in range(1, 11):
    # train

    optimizer_1.zero_grad()
    optimizer_1.step()
    print("第%d个epoch的学习率: %f"%(epoch, optimizer_1.param_groups[0]['lr']))
    scheduler_1.step()

初始化的学习率:  0.1
第1个epoch的学习率: 0.100000
第2个epoch的学习率: 0.100000
第3个epoch的学习率: 0.100000
第4个epoch的学习率: 0.010000
第5个epoch的学习率: 0.010000
第6个epoch的学习率: 0.010000
第7个epoch的学习率: 0.001000
第8个epoch的学习率: 0.001000
第9个epoch的学习率: 0.001000
第10个epoch的学习率: 0.000100


In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
import itertools

initial_lr = 0.1

class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

    def forward(self, x):
        pass

net_1 = model()

optimizer_1 = torch.optim.Adam(net_1.parameters(), lr=initial_lr)
scheduler_1 = MultiStepLR(optimizer_1, )

